# System overview data example

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/andrewm4894/netdata-gpt-notebooks/blob/main/notebooks/minimal_data_example.ipynb)

In [27]:
# if running in colab uncomment the following line and run it to install the required packages
#!pip install python-dotenv netdata-pandas openai

In [28]:
import os
from dotenv import load_dotenv
import pandas as pd
from netdata_pandas.data_cloud import get_data_cloud
import openai
import pprint as pp

# load tokens from .env file
load_dotenv()

NETDATA_API_TOKEN = os.getenv('NETDATA_API_TOKEN')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

openai.api_key = OPENAI_API_KEY

In [29]:
# inputs

# netdata demo space https://app.netdata.cloud/spaces/netdata-demo/rooms/host-reachability
space_id = 'ea93d7b8-0df6-45c0-b13d-1560996c89eb' 
room_id = 'bd9b0ab8-60a7-4ced-adfa-5c2e3cb196cd'
contexts = [
    'system.cpu',
    'system.load',
    'system.ip',
    'system.ram',
    'system.processes',
    'system.io',
    ]

# last 10 minutes of data
after = -60*10
before = 0
freq = '15s' # 10 seconds

In [30]:
df = pd.DataFrame(columns=['time'])

# get data from netdata cloud
for context in contexts:
    df_context = get_data_cloud(space_id, room_id, context, after, before, freq=freq)
    df_context = df_context.add_prefix(f'{context}.')
    df = df.merge(df_context,how='outer',on='time')

df_time_min = df.index.min()
df_time_max = df.index.max()

# save tokens by using t
df['t'] = range(len(df))
df = df.set_index('t')
del df['time']
# try save tokens by using ints
df = (df*100).round(0).ffill().bfill().astype('int')
print(df.shape)
df.head()

(40, 19)


,system.cpu.iowait,system.cpu.nice,system.cpu.softirq,system.cpu.steal,system.cpu.system,system.cpu.user,system.load.load1,system.load.load15,system.load.load5,system.ip.InOctets,system.ip.OutOctets,system.ram.buffers,system.ram.cached,system.ram.free,system.ram.used,system.processes.blocked,system.processes.running,system.io.in,system.io.out
t,,,,,,,,,,,,,,,,,,,
0,13,268,258,0,528,709,95,97,93,26571,50638,2294,414682,40552,319593,0,160,65896,3121
1,8,243,145,0,488,559,72,96,87,26849,51064,2296,414960,40264,319601,0,160,65896,3121
2,8,277,297,0,543,634,68,95,86,26163,50421,2298,415194,39816,319813,13,173,73419,29085
3,10,266,168,0,474,533,72,95,86,26485,50833,2300,415455,39593,319774,20,240,73275,3436
4,17,277,289,0,841,923,74,94,85,27444,50973,2302,415782,38788,320250,0,280,116102,50301


In [31]:
prompt = f"""
You are an experienced SRE and sysadmin.

You are monitoring an server using Netdata Cloud.

Below is the some system overview metrics for this server from {df_time_min} to {df_time_max}, aggregated to a {freq} frequency.

The metrics have all been multiplied by 100 and rounded to the nearest integer to be easier to work with.

Does this server have a problem?

Metrics data:
{df.to_string(max_rows=len(df), max_cols=(len(df.columns)))}
"""
print(prompt)


You are an experienced SRE and sysadmin.

You are monitoring an server using Netdata Cloud.

Below is the some system overview metrics for this server from 0 to 39, aggregated to a 15s frequency.

The metrics have all been multiplied by 100 and rounded to the nearest integer to be easier to work with.

Does this server have a problem?

Metrics data:
    system.cpu.iowait  system.cpu.nice  system.cpu.softirq  system.cpu.steal  system.cpu.system  system.cpu.user  system.load.load1  system.load.load15  system.load.load5  system.ip.InOctets  system.ip.OutOctets  system.ram.buffers  system.ram.cached  system.ram.free  system.ram.used  system.processes.blocked  system.processes.running  system.io.in  system.io.out
t                                                                                                                                                                                                                                                                                        

In [32]:
# build messages list to pass to openai
messages=[
    {"role": "user", "content": prompt}
]

In [33]:
# call openai api
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages,
)

In [34]:
reply_content = completion.choices[0].message.content
pp.pprint(reply_content)

('Based on the provided metrics, it does not seem like the server has a '
 'problem. The CPU usage is relatively low, the load average is within '
 'reasonable limits, and the RAM utilization is not high. Additionally, there '
 'are no processes blocked or processes in a running state, which suggests '
 'that the server is not experiencing any performance issues. The network '
 'traffic is also not exceptionally high. However, this is only a snapshot of '
 "the server's performance, and more detailed analysis would be necessary to "
 'make a definitive conclusion.')
